## 144_5 dashboard

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-10-05 12:07:18


In [2]:
# ---------------------- Build HTML dashboard --------------------------------

import html
import pandas as pd
from datetime import datetime
import time,os,sys,re
from pathlib import Path 
import shutil



def make_sat_dashboard(
    OUTPUT_DIR: str,
    PROJECT_NAME: str,
    stamp: str,
    *,
    map_gallery: list[dict] | None = None,
    intro_html: str = "",                 
    charts_blocks: list[dict] | None = None,
    summary_html: str = "",
    user_cases: list[dict] | None = None
) -> str:
    gallery_html = _render_map_gallery_html(map_gallery or [])
    intro_block  = _render_intro_html(intro_html)              
    charts_html  = _render_charts_html(charts_blocks or [])    
    user_cases_html = _render_user_cases_html(user_cases or []) 

    html_out = f"""
<!doctype html>
<html lang="en">
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <meta charset="utf-8">
  <title>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <style>
    body{{margin:0;padding:20px;background:#f8fafc;font:14px/1.4 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;color:#111827}}
    .sat-wrap{{max-width:1200px;margin:0 auto}}
    .sat-hero{{background:#111827;border-radius:14px;color:white;padding:16px 18px;margin-bottom:16px;display:flex;flex-wrap:wrap;align-items:center;gap:10px}}
    .sat-hero h1{{margin:0;font-size:20px}}
    .sat-hero a{{color:#a5b4fc;text-decoration:none}}
    .sat-row{{display:grid;gap:14px;grid-template-columns:1fr}}
    @media(min-width:960px){{ .sat-row{{grid-template-columns:2fr 1fr}} }}
    .sat-panel{{background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.06);padding:12px}}
    .muted{{color:#6b7280}}
    .sat-card{{ position:relative; }}
    .issues{{ position:absolute; top:10px; right:10px; display:flex; gap:6px; align-items:center; z-index:2; }}
    .issue-icon{{ font-size:18px; line-height:1; color:#374151; text-decoration:none; }}
    .issue-icon:hover{{ color:#111827; }}
    .issue-badge{{
      font:600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
      padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none; border:1px solid #e5e7eb;
    }}
    .issue-badge:hover{{ background:#e5e7eb; }}
  </style>
</head>
<body>
  <div class="sat-wrap">
    <div class="sat-hero">
      <h1>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</h1>
      <div class="muted" style="margin-left:auto">Updated: {html.escape(stamp)}</div>
    </div>

    {intro_block}     
    {gallery_html}
    {user_cases_html} 
    <div class="sat-row">
      <div class="sat-panel">
        {charts_html}   <!-- NEW: chart panels with descriptions -->
      </div>
      <div class="sat-panel">
        {summary_html}
      </div>
    </div>
  </div>
  </div>
  </div>
<script>
  // === FILTER TAGS SCRIPT START ===
  function updateFilters() {{
    const active = Array.from(document.querySelectorAll('.uc-tag.active'))
                        .map(t => t.dataset.tag.toLowerCase());

    document.querySelectorAll('.uc-card').forEach(card => {{
      const tags = card.dataset.tags.toLowerCase();
      if (active.length === 0 || active.some(a => tags.includes(a))) {{
        card.style.display = 'block';
      }} else {{
        card.style.display = 'none';
      }}
    }});
  }}

  function toggleTag(tagText) {{
    const selector = `.uc-tag[data-tag="${{tagText}}"]`;
    document.querySelectorAll(selector).forEach(tagEl => {{
      tagEl.classList.toggle('active');
    }});
    updateFilters();
  }}

  document.addEventListener('click', e => {{
    if (e.target.classList.contains('uc-tag')) {{
      const tagText = e.target.dataset.tag;
      toggleTag(tagText);
    }}
    if (e.target.id === 'reset-filters') {{
      document.querySelectorAll('.uc-tag.active').forEach(t => t.classList.remove('active'));
      updateFilters();
    }}
  }});
  // === FILTER TAGS SCRIPT END ===
</script>
</body>
</html>
"""

    # save + latest, same as before
    out_dir = Path(OUTPUT_DIR); out_dir.mkdir(parents=True, exist_ok=True)
    html_path = out_dir / f"{PROJECT_NAME}_dashboard_{stamp}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_out)
    latest_path = _to_latest_name(html_path)
    shutil.copyfile(html_path, latest_path)
    print(f"Saved: {html_path}\nUpdated: {latest_path}")
    print("Current path:", Path.cwd())
    return str(html_path)

def _render_intro_html(intro_html: str, mode: str = "on") -> str:
    """
    mode:
      - "off"         -> render nothing
      - "inline"      -> tiny inline link bar (no big panel, minimal spacing)
      - "collapsible" -> <details> with a 'Links' summary (closed by default)
      - "panel"       -> compact panel (if you still want a box)
    """
    if not intro_html or mode == "off":
        return ""

    if mode == "inline":
        return f'''
        <div class="sat-links" style="margin:4px 0 6px; font-size:.92rem;
             display:flex; flex-wrap:wrap; gap:.5rem; align-items:center;">
          {intro_html}
        </div>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:0 }}
        </style>
        '''

    if mode == "collapsible":
        return f'''
        <details class="sat-links" style="margin:2px 0">
          <summary style="cursor:pointer; font-weight:600">Links</summary>
          <div style="margin-top:6px; display:flex; flex-wrap:wrap; gap:.5rem;">
            {intro_html}
          </div>
        </details>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:.2rem 0 }}
        </style>
        '''

    # fallback: compact panel (kept for completeness)
    return f'''
    <section class="sat-panel" style="padding:6px 0">
      <div class="sat-intro">{intro_html}</div>
    </section>
    <style>
      .sat-intro p{{margin:.2rem 0}}
      .sat-intro a{{color:#2563eb;text-decoration:none}}
      .sat-intro a:hover{{text-decoration:underline}}
    </style>
    '''

def _render_charts_html(charts: list[dict]) -> str:
    if not charts:
        return ""
    import html as _h, re

    def _badge(url: str) -> str:
        # Try to extract issue number to show like #142
        m = re.search(r"/issues/(\d+)", url)
        label = f"#{m.group(1)}" if m else "Issue"
        u = _h.escape(url)
        return f'<a class="issue-badge" href="{u}" target="_blank" rel="noopener">{label}</a>'

    out = []
    for ch in charts:
        title  = _h.escape(ch.get("title", ""))
        body   = ch.get("body_html", "")
        issues = ch.get("issues") or []  # list of URLs (strings)
        badges = " ".join(_badge(u) for u in issues)

        out.append(f"""
        <section class="sat-panel">
          <div class="chart-head">
            <h3 class="chart-title">{title}</h3>
            {'<div class="issue-badges">'+badges+'</div>' if badges else ''}
          </div>
          <div class="chart-desc">{body}</div>
        </section>
        """)

    return """
    <div class="charts-col">
      {}
    </div>
    <style>
      .charts-col > .sat-panel + .sat-panel{{margin-top:12px}}

      .chart-head{{display:flex;align-items:center;gap:8px;flex-wrap:wrap}}
      .chart-title{{margin:0;font-size:16px;font-weight:700}}

      .issue-badge{{
        display:inline-block; padding:2px 8px; border-radius:999px;
        background:#eef2ff; color:#3730a3; font-size:12px; font-weight:600;
        text-decoration:none; border:1px solid #c7d2fe;
      }}
      .issue-badge:hover{{background:#e0e7ff}}
      .chart-desc{{color:#374151}}
      .chart-desc p{{margin:.4rem 0}}
      .chart-desc a{{color:#2563eb;text-decoration:none}}
      .chart-desc a:hover{{text-decoration:underline}}
    </style>
    """.format("".join(out))
    

In [3]:
def _render_map_gallery_html(items: list[dict]) -> str:
    """
    Render a responsive gallery of map links with:
      - 'issues': list[str] (GitHub issue URLs → badges)
      - 'buzzwords': list[str] (tags shown as chips + filter)
      - 'stakeholders': list[str] (tags shown as chips + filter)
      - 'desc_is_html': bool (whether desc contains raw HTML)
    """
    if not items:
        return ""

    import html as _h, re

    def _esc(v): 
        return _h.escape("" if v is None else str(v))

    # --- Issue badges helper ---
    def _issue_badges(issues: list[str] | None) -> str:
        if not issues:
            return ""
        badges = []
        for u in issues:
            m = re.search(r"/issues/(\d+)", u or "")
            num = m.group(1) if m else None
            label = f"#{num}" if num else "Issue"
            title = f"Issue #{num}" if num else "GitHub Issue"
            badges.append(
                f'<a class="issue-badge" href="{_esc(u)}" target="_blank" rel="noopener" '
                f'title="{_esc(title)}">{_esc(label)}</a>'
            )
        # GitHub icon linking to first issue
        badges.insert(
            0,
            f'<a class="issue-icon" href="{_esc(issues[0])}" target="_blank" '
            f'rel="noopener" title="GitHub Issues"><i class="fab fa-github"></i></a>'
        )
        return f'<div class="issues">{"".join(badges)}</div>'

    # --- Collect unique buzzwords + stakeholders ---
    buzzwords = sorted({bw for it in items for bw in it.get("buzzwords", [])})
    stakeholders = sorted({st for it in items for st in it.get("stakeholders", [])})

    # --- Chip helper ---
    def chip(label, tag, kind):
        return f'<button class="filter-chip {kind}" data-tag="{_esc(tag)}">{_esc(label)}</button>'

    # --- Build filter bar ---
    filter_html = ""
    if buzzwords or stakeholders:
        buzz_html = "".join(chip(b, b, "buzz") for b in buzzwords)
        stake_html = "".join(chip(s, s, "stakeholder") for s in stakeholders)
        filter_html = f"""
        <div class="filter-bar">
          <div class="filter-section"><strong>🏷️ Buzzwords:</strong> {buzz_html}</div>
          <div class="filter-section"><strong>👥 Stakeholders:</strong> {stake_html}</div>
          <button class="filter-clear" onclick="clearFilters()">Rensa filter</button>
        </div>
        """

    # --- Build cards ---
    cards = []
    for it in items:
        title = _esc(it.get("title", ""))
        url = _esc(it.get("url", ""))
        img1x = _esc(it.get("img", ""))
        img2x = _esc(it.get("img2x", "")) or img1x.replace("400x225", "800x450")
        desc_raw = it.get("desc", "")
        desc_html = desc_raw if it.get("desc_is_html", False) else _esc(desc_raw)

        buzz = it.get("buzzwords", [])
        stake = it.get("stakeholders", [])
        tags_for_data = ",".join(buzz + stake)

        buzz_tags_html = "".join(f'<span class="tag buzz">{_esc(t)}</span>' for t in buzz)
        stake_tags_html = "".join(f'<span class="tag stakeholder">{_esc(s)}</span>' for s in stake)

        issues = it.get("issues") if isinstance(it.get("issues"), list) else None
        issues_html = _issue_badges(issues)

        # --- Media block ---
        if img1x:
            media_html = f"""
              <a class="gallery-card-link" href="{url}" target="_blank" rel="noopener">
                <picture>
                  <source srcset="{img1x} 1x, {img2x} 2x" type="image/jpeg">
                  <img class="gallery-media" src="{img1x}"
                       srcset="{img1x} 1x, {img2x} 2x"
                       width="400" height="225" loading="lazy" decoding="async"
                       alt="{title}">
                </picture>
              </a>
            """
        else:
            media_html = f"""
              <a class="gallery-fallback" href="{url}" target="_blank" rel="noopener">
                <div class="gallery-fallback-title">{title}</div>
              </a>
            """

        # --- Card ---
        card = f"""
        <article class="sat-card" data-tags="{_esc(tags_for_data)}">
          {issues_html}
          {media_html}
          <div class="gallery-meta">
            <a class="gallery-title" href="{url}" target="_blank">{title}</a>
            {f'<div class="tag-row buzzwords">{buzz_tags_html}</div>' if buzz_tags_html else ''}
            {f'<div class="tag-row stakeholders">{stake_tags_html}</div>' if stake_tags_html else ''}
            <div class="gallery-desc">{desc_html}</div>
          </div>
        </article>
        """
        cards.append(card)

    # --- Full HTML ---
    return f"""
    <section class="sat-panel">
      <div class="gallery-header">
        <h2>Kartgalleri</h2>
        <div class="muted">Länkar till senaste skapade kartorna</div>
      </div>
      {filter_html}
      <div class="sat-gallery">
        {''.join(cards)}
      </div>
    </section>

    <style>
      /* --- Filter bar --- */
      .filter-bar {{ display:flex; flex-direction:column; gap:8px; margin-bottom:14px; }}
      .filter-section strong {{ margin-right:6px; font-weight:600; font-size:14px; }}

      .filter-chip {{
        font-size:13px; padding:4px 10px; border-radius:999px;
        border:1px solid #d1d5db; cursor:pointer;
      }}
      .filter-chip.buzz {{ background:#eef2ff; color:#3730a3; border-color:#c7d2fe; }}
      .filter-chip.stakeholder {{ background:#d1fae5; color:#065f46; border-color:#a7f3d0; }}
      .filter-chip.buzz.active {{ background:#2563eb; color:white; border-color:#2563eb; }}
      .filter-chip.stakeholder.active {{ background:#059669; color:white; border-color:#059669; }}

      .filter-clear {{
        font-size:13px; padding:4px 10px; border-radius:6px;
        border:1px solid #d1d5db; background:#fff; cursor:pointer; align-self:flex-start;
      }}

      /* --- Gallery --- */
      .sat-gallery {{ display:grid; gap:12px; grid-template-columns:1fr; }}
      @media (min-width:680px) {{
        .sat-gallery {{ grid-template-columns:repeat(2, minmax(0,1fr)); }}
      }}
      @media (min-width:1024px) {{
        .sat-gallery {{ grid-template-columns:repeat(3, minmax(0,1fr)); }}
      }}

      .sat-card {{
        position:relative;
        background:#fff; border-radius:12px; box-shadow:0 2px 10px rgba(0,0,0,.06);
        padding:10px; display:flex; flex-direction:column; gap:8px;
      }}

      /* Issues (top-right corner) */
      .issues {{
        position:absolute; top:10px; right:10px;
        display:flex; align-items:center; gap:6px; flex-wrap:wrap;
        z-index:2;
      }}
      .issue-icon {{
        font-size:18px; line-height:1; text-decoration:none; color:#374151;
      }}
      .issue-icon:hover {{ color:#111827; }}

      .issue-badge {{
        font: 600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
        padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none;
        border:1px solid #e5e7eb;
      }}
      .issue-badge:hover {{ background:#e5e7eb; }}

      .gallery-card-link {{ display:block; border-radius:10px; overflow:hidden; background:#f3f4f6; }}
      .gallery-media {{ display:block; width:100%; border-radius:10px; object-fit:cover; }}
      .gallery-fallback {{
        display:flex; align-items:center; justify-content:center;
        height:140px; border-radius:10px; background:#111827; color:#e5e7eb;
      }}
      .gallery-fallback-title {{ font-weight:600; }}

      .gallery-meta {{ display:flex; flex-direction:column; gap:4px; }}
      .gallery-title {{ font-weight:700; color:#111827; text-decoration:none; }}
      .gallery-title:hover {{ text-decoration:underline; }}
      .gallery-desc {{ color:#374151; font-size:13px; }}

      /* Buzzwords & Stakeholders */
      .tag-row {{ display:flex; flex-wrap:wrap; gap:6px; margin:4px 0; }}
      .tag {{
        font-size:12px; padding:2px 8px; border-radius:999px; border:1px solid #e5e7eb;
      }}
      .tag.buzz {{ background:#eef2ff; color:#3730a3; border-color:#c7d2fe; }}
      .tag.stakeholder {{ background:#d1fae5; color:#065f46; border-color:#a7f3d0; }}
    </style>

    <script>
      function clearFilters() {{
        document.querySelectorAll('.filter-chip').forEach(c => c.classList.remove('active'));
        document.querySelectorAll('.sat-card').forEach(c => c.style.display='');
      }}
      document.querySelectorAll('.filter-chip').forEach(chip => {{
        chip.addEventListener('click', () => {{
          chip.classList.toggle('active');
          const active = Array.from(document.querySelectorAll('.filter-chip.active'))
                              .map(c => c.getAttribute('data-tag'));
          document.querySelectorAll('.sat-card').forEach(c => {{
            const tags = (c.getAttribute('data-tags') || '').split(',');
            const match = active.every(a => tags.includes(a));
            c.style.display = active.length ? (match ? '' : 'none') : '';
          }});
        }});
      }});
    </script>
    """


In [4]:
def _render_user_cases_html() -> str:
    def _render_tags(tags: list[str], kind: str) -> str:
        # kind = "buzzword" or "stakeholder"
        out = []
        for t in tags:
            out.append(f'<a class="tag tag-{kind}" href="?{kind}={t}">{t}</a>')
        return " ".join(out)


    cards = []
    for c in cases:
        needs_html = "".join(f"<li>{n}</li>" for n in c["needs"])
        status_html = "".join(f"<li>{s}</li>" for s in c["status"])
        buzzwords_str = ",".join(c["buzzwords"])
        stakeholders_str = ",".join(c["stakeholders"])

        buzzwords_html = _render_tags(c["buzzwords"], "buzzword")
        stakeholders_html = _render_tags(c["stakeholders"], "stakeholder")

        card = f"""
        <article class="usercase-card"
                 data-buzzwords="{buzzwords_str}"
                 data-stakeholders="{stakeholders_str}">
          <h3>{c['icon']} {c['title']}</h3>
          <div class="usercase-section">
            <strong>Needs:</strong>
            <ul>{needs_html}</ul>
          </div>
          <div class="usercase-section">
            <strong>Status today:</strong>
            <ul>{status_html}</ul>
          </div>
          <div class="usercase-meta" style="font-size:12px">
            {buzzwords_html}<br>
            {stakeholders_html}
          </div>
        </article>
        """
        cards.append(card)

    return f"""
    <section class="sat-panel">
      <div class="gallery-header">
        <h2>User Cases</h2>
        <div class="muted">Different users and their needs along the SAT</div>
      </div>
      <div class="usercase-grid">
        {''.join(cards)}
      </div>
    </section>

    <style>
      .usercase-grid {{
        display:grid; gap:16px; grid-template-columns:1fr;
      }}
      @media (min-width:800px) {{
        .usercase-grid {{ grid-template-columns:repeat(2, minmax(0,1fr)); }}
      }}
      .usercase-card {{
        background:#fff; border-radius:12px; box-shadow:0 2px 8px rgba(0,0,0,.08);
        padding:14px; display:flex; flex-direction:column; gap:10px;
      }}
      .usercase-card h3 {{
        margin:0; font-size:16px; font-weight:700; display:flex; align-items:center; gap:6px;
      }}
      .usercase-section strong {{
        font-size:13px; color:#374151;
      }}
      .usercase-section ul {{
        margin:4px 0 0; padding-left:20px; font-size:13px; color:#374151;
      }}
      .usercase-section li {{
        margin-bottom:3px;
      }}
      .tag {{
        display:inline-block; margin:2px; padding:2px 8px;
        border-radius:999px; font-size:12px; font-weight:600;
        text-decoration:none;
      }}
      .tag-buzzword {{
        background:#eef2ff; color:#3730a3; border:1px solid #c7d2fe;
      }}
      .tag-buzzword:hover {{ background:#e0e7ff; }}
      .tag-stakeholder {{
        background:#dcfce7; color:#166534; border:1px solid #bbf7d0;
      }}
      .tag-stakeholder:hover {{ background:#bbf7d0; }}
    </style>

    <script>
      const params = new URLSearchParams(window.location.search);
      const buzzword = params.get("buzzword");
      const stakeholder = params.get("stakeholder");

      if (buzzword || stakeholder) {{
        document.querySelectorAll('.usercase-card').forEach(el => {{
          const buzz = el.dataset.buzzwords.split(",");
          const stks = el.dataset.stakeholders.split(",");
          const buzzMatch = !buzzword || buzz.includes(buzzword);
          const stkMatch = !stakeholder || stks.some(s => s.includes(stakeholder));
          if (!(buzzMatch && stkMatch)) {{
            el.style.display = "none";
          }}
        }});
      }}
    </script>
    """


In [5]:
def _render_user_cases_html(user_cases: list[dict]) -> str:
    if not user_cases:
        return ""

    def _tag(text, href="#"):
        return f'<a class="uc-tag" href="{href}" target="_blank" rel="noopener">{text}</a>'

    out = []
    for case in user_cases:
        buzz = " ".join(_tag(b) for b in case.get("buzzwords", []))
        stakeholders = " ".join(_tag(s) for s in case.get("stakeholders", []))
        desc = case.get("desc", "")
        out.append(f"""
        <div class="uc-card">
          <div class="uc-title">{case['title']}</div>
          <div class="uc-meta"><b>Keywords:</b> {buzz}</div>
          <div class="uc-meta"><b>Stakeholders:</b> {stakeholders}</div>
          <div class="uc-desc">{desc}</div>
        </div>
        """)

    return """
    <section class="sat-panel">
      <h2>User Cases</h2>
      <div class="uc-grid">{}</div>
    </section>
    <style>
      .uc-grid {{
        display:grid;
        grid-template-columns:repeat(auto-fit,minmax(280px,1fr));
        gap:18px;
        margin-top:12px;
      }}
      .uc-card {{
        border:1px solid #e5e7eb;
        border-radius:12px;
        padding:14px;
        background:#fff;
        box-shadow:0 2px 4px rgba(0,0,0,0.05);
      }}
      .uc-title {{
        font-weight:600;
        font-size:15px;
        margin-bottom:6px;
      }}
      .uc-meta {{
        font-size:13px;
        color:#374151;
        margin-bottom:4px;
        display:flex;
        flex-wrap:wrap;
        gap:6px;
      }}
      .uc-tag {{
        display:inline-block;
        padding:4px 10px;
        border-radius:999px;
        font-size:12px;
        font-weight:500;
        text-decoration:none;
      }}
      .uc-meta:nth-of-type(1) .uc-tag {{
        background:#eef2ff;
        color:#3730a3;
      }}
      .uc-meta:nth-of-type(2) .uc-tag {{
        background:#dcfce7;
        color:#166534;
      }}
      .uc-tag:hover {{
        opacity:0.85;
      }}
    </style>
    """.format("".join(out))


In [6]:
# =================== Save helpers ===========================================
def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r'_(?:20\d{6})(?:_\d{4})?$', '', stem)
    return p.with_name(new_stem + "_latest.html")

def save_with_latest(map_object, filename: str | Path):
    from pathlib import Path

    print("Current path:", Path.cwd())
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    print("Latest path ",latest_path)
    shutil.copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")


In [25]:
gallery = [
    {
        "title": "👉 📜 Machine-translated regulations for nature reserves along SAT",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_195_MachineTranslate_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT195_MachineTranslation_400x225.jpg",
        "desc": "📜 One key stakeholder for SAT is visitors from abroad, who need to understand "
        "the nature reserve regulations that are currently published only in Swedish. "
        "This map use Google Translate for the most common languages spoken in Sweden and by tourist"
        "<br /><br /><a href='https://youtu.be/4MmD1EctW2Y' target=_blank>▶️ video</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/195",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/78"
        ],
        "buzzwords": ["Ekosystem", "Flerspråkighet","Smart tourism"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare","Visit Sweden" ]        
    },  
    {
        "title": "🖼️ RAÄ Bebyggelseregistret - SAT",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_BBR_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_BBR_220_400x225.jpg",
        "desc": "🖼️ Example of using BBR to discover interesting points of interest (POIs) along the SAT trail."
        "<br /><br/>* <a target=_blank href='https://www.raa.se/hitta-information/oppna-data/oppna-data-portal/'>Öppna data</a>"
        "<br />* <a target=_blank href='https://www.raa.se/hitta-information/bebyggelseregistret/'>Bebyggelseregistret (BBR)</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/220"
        ],
        "buzzwords": ["Smart tourism"],
        "stakeholders": ["Hikers"]        
    },     
    {
        "title": "👉 📜 SAT / Wikidata / OSM / Social media / Opening hours",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_217_WIkipedia_OSM_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_OSM_WD_217_400x225.jpg",
        "desc": "📜 This prototype shows how the Stockholm Archipelago Trail can be managed in a data-driven way using Linked Open Data. "
        "By connecting Wikidata, OSM, and multilingual resources, regulations and trail information can be dynamically presented not only in Swedish but also in the languages most spoken by visitors. "
        "<br/><br/>Such an approach makes it possible to link content directly to Google Maps, social media platforms, and other digital services, enabling smarter tourism experiences. "
        "<br /><br /><a href='https://youtu.be/_nbI8hkRAvA' target=_blank>▶️ video</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/217",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["APIFirst", "Flerspråkighet","Linked data", "Smart tourism"],
        "stakeholders": ["Community", "Non-Swedish speaking hikers", "Vandrare"]        
    },    {
        "title": "🚻 Toaletter nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_132_toilets.jpg",
        "desc": "🚻 Söker i OpenStreetMap efter toaletter längs leden och ser vilket metadata "
        "som saknas. Idag saknas realtidsinfo och kopplingar till aktuell status – trots att "
        "Skärgårdsstiftelsen fått 1 miljon Euro för digitalisering."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>Toaletter som friluftslivsdata</b> som skall levereras"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/127",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas","Ekosystem", "Flerspråkighet","Smart tourism", "Öppen data"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare", ]        
    },
    {
        "title": "💧 Dricksvatten nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_139_drinkingwater.jpg",
        "desc": "💧 Söker i OpenStreetMap efter dricksvatten längs leden. "
        "Idag saknas ett ekosystem där man digitalt kan se vad som är påslaget och testat. "
        "Öppna data från Skärgårdsstiftelsen borde ge exempel på hur skärgården beskrivs digitalt."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>tillgång till dricksvatten som friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas", "Ekosystem", "Flerspråkighet","Interoperabilitet","Öppen data"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare", ]        
    },
    {
        "title": "🖼️ Koppling mellan led och bild – Image Quality Control",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Image_Quality_Control_400x225.jpg",
        "desc": "🖼️ Hela leden är dokumenterad i OSM där varje etapp "
        "och delsträcka kopplas till bilder på Wikimedia Commons. "
        "Denna karta låter dig kvalitetssäkra både kartinfo och bildmaterial. "
        "Innehåller lager för 🚻 toaletter och 💧 dricksvatten."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ bilddatabaser med fria bilder och länkade data</li></ll>",
        "desc_is_html": True,
        "issues": ["https://github.com/salgo60/ProjectOutdoorGyms/issues/74",
                  "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["APIFirst","Bilddatabas","Ekosystem", "Interoperabilitet", "Öppen data"],
        "stakeholders": ["Naturvårdsverket"]        
    },
    {
        "title": "♿ Funktionstillgänglighet – rullstol",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_WHEELCHAIR_073_wheelchair_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Wheelchair_thumbnail.jpg",
        "desc": "♿ Visar objekt längs leden som är taggade 'wheelchair' i OSM. "
        "Även här efterlyses bättre öppna data från Skärgårdsstiftelsen. "
        "<strong>Naturvårdsverket</strong> som alla tror är expertmyndighet ser vi har funderat sedan 2019. "
        "När till och med Myndigheten för delaktighet sitter med i Myndighetsnätverket om vandringsleder men man sedan 2019 inte lyckats "
        "ställa tydliga krav på tillgänglighet eller leverera data som beskriver detta – då är det uppenbart att det är fel laguppställning. "
        "Fortsätter det så här kommer ingenting att vara på plats ens 2035. Hur svårt kan det vara... med chatGPT 5 sekunder.."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/172",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174"
        ],
        "buzzwords": ["Flerspråkighet","Ekosystem", "Interoperabilitet","Tillgänglighet", "Öppen data"],
        "stakeholders": ["Naturvårdsverket", "Vandrare", "Skärgårdsstiftelsen"]        
    },
    {
        "title": "✅ Att göra i OSM – Todo",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_ALL_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "✅ Visualiserar saknade OSM-taggar på en karta över leden.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet", "Öppna API:er"],
        "stakeholders": ["Community","Naturvårdsverket"]        
    },
    {
        "title": "🔍 Audit-lager i OSM",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "🔍 Karta som visar var OSM-taggar saknas för surface / foot / sac / trail_visibility / step_count.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet","Öppna API:er", "Öppen data"],
        "stakeholders": ["Community","Naturvårdsverket"]        
    },
    {
        "title": "🪜 Trappsteg på leden – Steps",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_steps_only_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Steps_Only_thumbnail.jpg",
        "desc": "🪜 SAT-leden har begärt ersättning för ~200 m trappor "
        "– men de är svåra att hitta. Är det detta som är överraskningen att inget levereras?",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas", "Flerspråkighet","Interoperabilitet","Tillgänglighet"],
        "stakeholders": ["Tillväxtverket", "Vandrare"]        
    },
    {
        "title": "🌊 Närhet till vatten – Proximity",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_water_proximity_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Water_Proximity_800x450.jpg",
        "desc": "🌊 Vissa sträckor går långt från vattnet – på gott och ont. Kartan visar hur nära/långt från vattnet olika delar ligger, samt längsta distans från vatten.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "Interoperabilitet", "Smart tourism","Öppen data"],
        "stakeholders": ["Vandrare"]        
    },
    {
        "title": "📚 Wikipedia/Wikidata",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_Wikipedia.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_Wikipedia_400x225.jpg",
        "desc": "📚 Mer än 600 Wikipedia/Wikidata-objekt har kopplats till SAT-leden. "
        "Tanken är att visa på hur <strong>digital interoperabilitet</strong> fungerar i praktiken, "
        "och hur man 2025 enkelt kan leverera flerspråkighet med vettiga ekosystem som hanterar"
        "persistenta identifierare och api:er <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🛰️ Länkade data samma som jmf <a href='https://github.com/salgo60/NOSAD-POC-Wikidata/issues/13' target=_blank >hur vi jobbar med Nobelprize.org</a> och uppdaterar Wikipedia via Wikidata</li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/22",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "APIFirst", "Community-driven", "Non-Swedish speaking hikers",  "Flerspråkighet", "Interoperabilitet", "Smart tourism","Öppen data", "Öppna API:er"],
        "stakeholders": ["Vandrare"]        
    },
    {
        "title": "🪧 SAT infoskyltar",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/182_WD_OSM_signs_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_InfoSign_400x225.jpg",
        "desc": "🪧 135 turistinformationsskyltar beställdes via Tillväxtverket – endast ~80 hittade. "
        "Ingen öppen data om placering eller leveransstatus. "
        "Vi skall överraskas - ja vi är överraskade hur dumt skattepengar slösas "
        "<i>The Magic slöseri</i><br/>"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet - följ internationell klassificering</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder på alla skyltarna som har unika id:n</li>"
        "<li>⏱️ information om tid att vandra sträckan</li>"
        "<li>⚠️ status på leden med senaste problem näs via QRC"
        "<li>📱 länk till sida som känner av språkinställning i mobilen och visar info på rätt språk"
        "<li>🍴🏨🛒🚰🚻 länk till sida som visar <b>öppna</b>restauranger, boenden, affärer, dricksvatten, toaletter"
        "</ll>"        ,
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/97",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/176",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/180",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas", "Flerspråkighet","Interoperabilitet", "QR-koder", "Smart tourism","Öppen data"],
        "stakeholders": ["Community","Tillväxtverket", "Vandrare"]        
    },
    {
        "title": "🐾 SAT – iNaturalist",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/179_inat_taxa_layers_colored_5.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_179_2_iNaturalist_400x225.jpg",
        "desc": "🐾 Datadriven test: på några minuter hämtas communitydriven artdata från iNaturalist och visar vad som faktiskt finns längs leden. Kontrasten mot långsamma processer är tydlig.",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148"
        ],
        "buzzwords": ["APIFirst", "Community-driven", "Ekosystem",  "Flerspråkighet" , "Non-Swedish speaking hikers" ,"Realtidsdata", "Smart tourism", "Öppna API:er", "Öppen data"],
        "stakeholders": ["Community", "Vandrare"]        
    },
    {
        "title": "🔥🏕️ SAT grillplatser & vindskydd",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/map185_vindskydd_grillplatser.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT188_WD_OSM_bbq_400x225.jpg",
        "desc": "🔥🏕️ 8 grillplatser och 11 vindskydd beställdes av Tillväxtverket för 7 miljoner kronor "
        "– men var är de och när levereras de? Kartan visar de som hittats längs leden.<br />"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🧑‍🤝‍🧑🗂️ digital koppling uppdrag, faktura, utbetalt belopp, leverabel. Idag visar Tillväxtverket ingen koppling beställning leverans ..."
        "Ifallet SAT blev det lunch på Grinda och en båttur Utö.. och att Tillväxtverket själva skrev en PDF... SAT själva säger att allt skall vara en överraskning...<br/>"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🔥 koppling skall finnas mellan beställning leverans</li>"
        "<li>🔥 projekt som SAT som inte redovisar begär tillbaka pengarna</li>"
        "<li>🛰️ API för felanmälan av alla objekt där objekten skall ha QRC kod, eller objekt 'nära mig'</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder och 'depict'</li></ll>",
        "desc_is_html": True,        
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185"
        ],
        "buzzwords": ["Bilddatabas","Flerspråkighet","Interoperabilitet", "Smart tourism", "Öppen data"],
        "stakeholders": ["Community", "Vandrare"]        
    },
    {
        "title": "🧭 SAT-leden och Nordsydlinjen",
        "url": "https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true#8/59.305/18.515",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_NordSyd_400x225.jpg",
        "desc": "🧭 Flum som <i>Magic season</i> möter verklighet<br/> SAT-projektet hävdar att NordSyd-linjen ska göra det enkelt att vandra Stockholm Archipelago Trail. <strong>Sanningen</strong>? Nja. Få leder är så splittrade och otydliga som SAT. På kartan ser du hur NordSydlinjen faktiskt går – och var SAT-leden ligger. Och för att krångla till ekvationen ännu mer: SAT vill att vi ska vandra off-season, när NordSydlinjen inte ens går… <br/><b>Resultat:</b> dyra investeringar + flummiga löften = en led som aldrig riktigt hänger ihop. <strong>Tillväxtverket – vakna!</strong> Det handlar om våra skattepengar, som inte skapar verklig nytta utan göder ett osunt bidragstiggeri. Man pratar om “Magic season” och att vi ska “överraskas” – men det vi i praktiken överraskas av är hur oproffsigt skattepengar delas ut till lycksökare som levererar flum istället för resultat.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["APIFirst", "Ekosystem", "Öppen data", "Flerspråkighet", "Smart tourism"],
        "stakeholders": [ "Vandrare","Visit Sweden"]        
    },    
    {
        "title": "🏕️🌿 SAT – Campsites & Nature Reserves",
        "url": "https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true#10/59.0261/18.7015",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Naturreserve_400x225.jpg",
        "desc": "🏕️🌿 Skall man tälta på öarna gäller allemansrätten – "
        "om man inte är i ett naturreservat. 📜 Denna karta visar "
        "Stockholm Archipelago Trail, Naturreservat och länkar till Naturvårdsverkets Skyddad natur och Länsstyrelsens föreskrifter"
        ". Allt detta är på svenska – skall vi <strong>tro att SAT-projektet och Visit Sweden kommer</strong> att skapa en flerspråkig version av dessa sidor? 😉"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>"
        "<br /><br /><b>Interoperabiltet</b> OSM <-> Wikdata <-> Wikicommons",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["Bilddatabas", "Community-driven", "Non-Swedish speaking hikers" ,"Smart tourism", "Öppen data", "Flerspråkighet"],
        "stakeholders": ["Vandrare","Visit Sweden"]        
    },
    {
        "title": "🏕️🌿 SAT – Grillplatser.nu ",
        "url": "https://grillplatser.nu/Karta/Lan/Stockholms-lan",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_grillplatsernu_400x225.jpg",
        "desc": "🏕️🌿 Öppna data har efterfrågats från <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11' target=_blank>SAT 2025-03-19</a> utan svar se hur > 7000 grillplatser "
        "finns samlade av en community med fria bilder. <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🌍 samma som grillplatser.nu OSM Wikidata</li>"
        "<li>🖼️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "</ll>"
        "<br /><br /><b>Interoperabiltet</b><br/> OSM <-> Wikdata <-> Grillplatser.nu <-> Wikicommons"
        "<ll><li>OSM <a href='https://wiki.openstreetmap.org/wiki/Sv:Key:ref:grillplatser.nu' "
        "target='_blanket'>Key:ref:grillplatser.nu</a></li>"
        "<li>OSM overpass <a href='https://overpass-turbo.eu/s/2cmU' "
        "target='_blanket'>Key:ref:grillplatser.nu</a> </li>"
        "</ll><br />",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/188"

        ],
        "buzzwords": ["Bilddatabas", "Community-driven", "Flerspråkighet", "Interoperabilitet",  "Smart tourism", "Öppen data"],
        "stakeholders": ["Vandrare"]        
    },
    {
        "title": "🔥 SAT – Wikidata - Wikipedia hur man jobbar datadrivet",
        "url": "https://wikishootme.toolforge.org/#lat=59.0617461708114&lng=18.431606590747837&zoom=12",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Wikishootme_400x225.jpg",
        "desc": "🔥 Öppna data från SAT lovas till <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49'>Nacka kommun</a> "
        "trovärdigheten är låg när allt är hemligt... ingen hittar vindskydd som utlovats, trappor som byggts..."
        "<br><br><a target=_blank href='https://www.youtube.com/watch?v=A3GnO4kAIos&list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R&index=1'>Video</a> "
        "om hur wikidata i kartan jobbar med bilder, bildbibliotek, > 200 språk versioner. "
        "<br /><br>Idag jobbar Google med <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' target=_blank>Persistenta Identifierare</a> och realtidsinformation med färjetider skall man "
        "prata om <b>Smart turism</b> fungerar det inte bara med filmer på Instagram om <i>Magic season</i>. "
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/143",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/145",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        
        ],
        "buzzwords": ["APIFirst","Interoperabilitet" , "Non-Swedish speaking hikers", "Smart tourism",],
        "stakeholders": ["Community"]        
    },
        {
        "title": "🔥 SAT – Ferrystops - datadrivet",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/187_WD_OSM_ferry_stops_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_ferry_400x225.jpg",
        "desc": "🔥 Öppna data från SAT dyker inte upp eftersom projektet inte arbetar datadrivet dom tror <i>det är ett vinterjobb</i>. " 
        "Var SAT leden startar och slutar är otydligt utan den knyter ihop bryggor och stigar och är inte en rundslinga. "
        "Den borde vara ett <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161'>Nodnätverk</a>. "
        "<br />Vi ser hela kedjan är pdf och textdokument från Tillväxtverket till utbetalning av skattepengar... och "
        "slutar med filmer på Instagram med <i>Magic season</i> för att sedan sökas nya bidrag..."
        "<br><b>Google</b> förstår detta med datadrivet och <b>Smart turism</b>. Dom jobbar <b>datadrivet</b> med Google Map och har realtidsinformation om alla "
        " Waxholmsbolagets bryggor och har även alla restauranger med recensioner... "
        "<br />Kartan ovan visar färjestopp som är startpunkter för SAT leden hämtat från <a target_blank href='https://www.youtube.com/watch?v=m_9_23jXPoE'>Wikidata</a>."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ger Tillväxtsverket pengar till datasilos <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/201",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/39",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/158",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/186",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/187"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Non-Swedish speaking hikers", "Realtidsdata", "Smart tourism"],
        "stakeholders": ["Skärgårdsstiftelsen","Vandrare"]        
    },
    
        {
        "title": "🔥 SAT – AED - Hjärtstartare <-> Hjärtstartarregistret",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/191_WD_OSM_AED_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_AED_191_400x225.jpg",
        "desc": "Hjärtstartare är till viss del uppstyrt av <a href='https://www.hjartstartarregistret.se/#/' target=_blank>Hjärtstartarregistret</a>  men lång ifrån alla apparater finns där. "
        "Jag har inventerat en del genom dels kolla i registret men även pratat med månniskor längs leden och i OSM."
        "<br><ll><li>Bilder på <a href='https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail_AED' target=_blank>Hjärtstartare längs leden</a> "
         "/ <a href='https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail_AED&subcats=true&subcatdepth=3&cluster=false' target=_blank> på en karta</a>"
        "<li>Jag har översatt en polsk app till svenska <a href='https://openaedmap.org/sv/#map=8.89/59.2745/18.942' target=_blank>OpenAED</a> som hämtar sitt data från OSM om AED:er "
        "se issue <a target=_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2820021924'>#46</a>" 
        "<li>Haft diskussion 2025-juni-19 13:30 med <a href='https://www.hjartstartarregistret.se/#/' target=_blank>hjärtstartarregistret</a> om bättre integration med OSM <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/79#issuecomment-2955570965'>#79</a>... "
        "status vi väntar på dom - tveksamt om dom har resurser"
        "<li>Skapat poster i OSM och Wikidata för AED hittade se <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191'>#191</a>"
        "</lL>"
        "<br><br><b>Utmaningar jag ser</b>"
        "<ll><li>Något som borde vara självklart att ha ordning på drivs av en organisation <a href='https://www.hlr.nu/'>HLR rådet</a>"
        "<li>var i <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2833502212' target=_blank>april i Nice</a> där AED:er finns som staten ansvarar för"
        " känns mycket bättre och <a href='https://openaedmap.org/sv/#map=12.71/43.68708/7.2744&node_id=12902177339' target=_blank>AED:er fanns på strandpromenaden</a>"    
        "<li>Hjärtstartarregistret använder idag Open Street Map kartor och skulle vinna på "
        "bättre integration men är nog inte datadrivna och har små resurser"
        "<li><b>SAT projektet har inte ens en projektyta</b> så vad dom vill berättas ostrukturerat på instagram meddelanden... - galet"
        "<li><b>Naturvårdsverket har projektet sedan 2019 om vandringsleder</b> som definierat när man skall mötas "
        "inte hur data om friluftsliv skall beskrivas... - galet se "
        "<a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144' target=_blank>ChatGPT om nationell samverkansmodell vandringsutveckling</a>"
        "</lL>"
        "<br />Saker som detta måste styras upp men jag ser ingen som kan detta... "
        "ser inte heller att Tillväxtverket bidrar med krav till Naturvårdsverket om bättre info"
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>hjärtstartare för friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Non-Swedish speaking hikers" , "Smart tourism"],
        "stakeholders": ["Vandrare"]        
    },

        {
        "title": "🔥 Wikipedia Recent Changes Map - real time",
        "url": "http://rcmap.hatnote.com/#sv,wikidata,en,uk",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/WikipediaChangeStream_400x225.jpg",
        "desc": """Exempel hur en community som Wikipedia jobbar med data som data<br/><br/>
        🚵 <b>Behov:</b> Att kunna se ändringar för alla vandringsleder i Sverige<br>
        ❌ <b>Idag:</b> Datasilos där vi inte ens vet vilka leder som finns i Sverige. Enormt frustrerande för vandrare...<br>
        ✅ <b>Önskat:</b> Datadrivet.
        """,
        "desc_is_html": True,
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Smart tourism"],
        "stakeholders": ["Community","Vandrare"]        
    },
        {
        "title": "🔥 OSM latest Changes ",
        "url": "https://rene78.github.io/latest-changes/#12/58.9388/18.1906",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/OSM_changeset_400x225.jpg",
        "desc": """Exempel hur OSM ändringar kan hämtas via en karta<br/>
        🚵 <b>Behov:</b> Att kunna se vem som ändrat vad för vandringsleder i Sverige<br>
        ❌ <b>Idag:</b> Datasilos verkar som Lantmäteriverket inte har ett vandringsleds fokus. Naturvårdsverket har inget APIfirst tänk utan mer likes på Linked in<br>
        ✅ <b>Önskat:</b> Datadrivet
        """,
        "desc_is_html": True,
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Smart tourism"],
        "stakeholders": ["Community","Vandrare"]        
    },

    
]



In [26]:
user_cases = [
      {
    "title": "Local business owner",
    "buzzwords": ["Economy", "Local growth", "Digital visibility"],
    "desc_is_html": True,
    "desc": """
    🏪 <b>Behov:</b> Synlighet på kartor, koppling till närliggande leder, 
    enkel uppdatering av öppettider och erbjudanden.<br>
    ❌ <b>Idag:</b> Många lokala företag finns inte på OSM/Wikidata 
    eller har felaktig info.<br>
    ✅ <b>Önskat:</b> Enkel onboarding till öppna data + integration med SAT Dashboard 
    så att fler hittar till lokala caféer, gårdsbutiker och vandrarhem.
    """
  },
  {
    "title": "Daytrip hikers",
    "buzzwords": ["Accessibility", "Digital twin","Transport"],
    "desc_is_html": True,
    "desc": """
    🚶‍♀️ <b>Behov:</b> Easy access trails for a day, clear maps, toilets and picnic areas, 
    public transport connections.<br>
    ❌ <b>Idag:</b> Hard to combine SL/ferry schedules with trail planning.<br>
    ✅ <b>Önskat:</b> Integrated map with public transport + SAT entry points.
    """
  },
  {
    "title": "Weekend tenting hikers",
    "buzzwords": ["Camping", "Transport"],
    "desc_is_html": True,
    "desc": """
    ⛺ <b>Behov:</b> Places for tents, drinking water, firewood/fireplaces, 
    ferry/boat access for a 2–3 day trip.<br>
    ❌ <b>Idag:</b> Wind shelters and water sources partly missing in OSM/Wikidata.<br>
    ✅ <b>Önskat:</b> Up-to-date camping data, clear rules for fire/tenting in nature reserves.
    """
  },
  {
    "title": "Good food & exclusive stay hikers",
    "buzzwords": ["Accommodation", "Smart tourism","Transport"],
    "desc_is_html": True,
    "desc": """
    🍷 <b>Behov:</b> Hiking combined with gourmet food, hotels, cabins, 
    high-quality experiences.<br>
    ❌ <b>Idag:</b> Few booking links or curated routes available.<br>
    ✅ <b>Önskat:</b> Packages combining SAT stages with local restaurants and stays.
    """
  },
  {
    "title": "Group hiking organizers",
    "buzzwords": ["Interoperability", "Planning"],
    "desc_is_html": True,
    "desc": """
    👥 <b>Behov:</b> Planning for larger groups (school classes, companies), 
    info about shelters, toilets, safety, group permissions.<br>
    ❌ <b>Idag:</b> Trail info is not adapted for group logistics.<br>
    ✅ <b>Önskat:</b> Group-friendly booking & facilities overview.
    """
  },
  {
    "title": "Hikers with boat access",
    "buzzwords": ["Transport", "Service"],
    "desc_is_html": True,
    "desc": """
    ⛵ <b>Behov:</b> Combine hiking with private boat access, 
    info on harbors, docks, overnight possibilities.<br>
    ❌ <b>Idag:</b> Transport info split across websites and social media.<br>
    ✅ <b>Önskat:</b> Unified harbor & service map linked to trail sections.
    """
  },
  {
    "title": "Non-Swedish speaking visitors",
    "buzzwords": ["Multilingualism", "Smart tourism", "Accessibility"],
    "desc_is_html": True,
    "desc": """
    🌍 <b>Behov:</b> Multilingual maps, translated regulations, 
    booking links for ferries & cabins, clear signage in English/German.<br>
    ❌ <b>Idag:</b> Regulations and maps mostly only in Swedish, 
    booking rarely linked.<br>
    ✅ <b>Önskat:</b> Machine-translated regulations, multilingual Wikidata/OSM labels, 
    integrated booking links.
    """
  },
  {
    "title": "MTB (mountain bike) riders",
    "buzzwords": ["Sport", "Transport"],
    "desc_is_html": True,
    "desc": """
    🚵 <b>Behov:</b> Clear rules about where MTB is allowed, trail surfaces, 
    alternative routes.<br>
    ❌ <b>Idag:</b> Few MTB routes tagged in OSM, unclear restrictions in nature reserves.<br>
    ✅ <b>Önskat:</b> MTB-friendly SAT map with allowed/forbidden areas.
    """
  },
    {
    "title": "Community sites (ideella föreningar, hembygdsföreningar)",
    "buzzwords": ["Culture", "Community", "Heritage"],
    "desc_is_html": True,
    "desc": """
    🏡 <b>Behov:</b> Visa hembygdsplatser, föreningshus, kulturarv, 
    lokala aktiviteter längs leden.<br>
    ❌ <b>Idag:</b> Spritt på olika små webbsidor, sällan integrerat i större kartor.<br>
    ✅ <b>Önskat:</b> Koppling till Wikidata/OSM så att föreningar 
    kan bli synliga direkt i SAT Dashboard och på turistkartor.
    """
  }, {
    "title": "Commercial tourist sites (Airbnb, glamping, aktivitetsbolag)",
    "buzzwords": ["Commercial", "Tourism", "Smart booking"],
    "desc_is_html": True,
    "desc": """
    🏕️ <b>Behov:</b> Länka boenden och aktiviteter direkt till leden, 
    ex. Airbnb-stugor, kajakuthyrning, guidade turer.<br>
    ❌ <b>Idag:</b> Data ligger låst i plattformar utan öppen länkning.<br>
    ✅ <b>Önskat:</b> Smarta länkar från trail-sektioner till boende/aktiviteter, 
    integration med bokningssystem och öppna API:er.
    """
  }
    
]


In [27]:
def _render_user_cases_html(user_cases: list[dict]) -> str:
    if not user_cases:
        return ""

    def _tag(text, cls):
        return f'<span class="uc-tag {cls}" data-tag="{text}">{text}</span>'

    out = []
    for case in user_cases:
        buzz = " ".join(_tag(b, "buzz") for b in case.get("buzzwords", []))
        desc = case.get("desc", "")
        tags_all = case.get("buzzwords", []) + case.get("stakeholders", [])
        tag_data = " ".join(tags_all)
        out.append(f"""
        <div class="uc-card" data-tags="{tag_data}">
          <div class="uc-title">{case['title']}</div>
          <div class="uc-meta"><b>Keywords:</b> {buzz}</div>
          <div class="uc-desc">{desc}</div>
        </div>
        """)

    return """
    <section class="sat-panel">
      <h2>User Cases</h2>
      <div class="uc-grid">{cards}</div>
    </section>

    <style>
      .uc-grid {{
        display:grid;
        grid-template-columns:repeat(auto-fit,minmax(280px,1fr));
        gap:18px;
        margin-top:12px;
      }}
      .uc-card {{
        border:1px solid #e5e7eb;
        border-radius:12px;
        padding:14px;
        background:#fff;
        box-shadow:0 2px 4px rgba(0,0,0,0.05);
        transition:all 0.2s ease;
      }}
      .uc-title {{
        font-weight:600;
        font-size:15px;
        margin-bottom:6px;
      }}
      .uc-meta {{
        font-size:13px;
        margin-bottom:4px;
        display:flex;
        flex-wrap:wrap;
        gap:6px;
      }}
      .uc-tag {{
        display:inline-block;
        padding:4px 10px;
        border-radius:999px;
        font-size:12px;
        font-weight:500;
        cursor:pointer;
        user-select:none;
      }}
      .uc-tag.buzz {{ background:#eef2ff; color:#3730a3; }}
      .uc-tag.active {{ outline:2px solid #111827; }}
    </style>

    <script>
      document.querySelectorAll('.uc-tag').forEach(tag => {{
        tag.addEventListener('click', () => {{
          // toggle active state
          tag.classList.toggle('active');
          // collect active tags
          const active = Array.from(document.querySelectorAll('.uc-tag.active'))
                              .map(t => t.dataset.tag.toLowerCase());
          // filter cards
          document.querySelectorAll('.uc-card').forEach(card => {{
            const tags = card.dataset.tags.toLowerCase();
            if (active.length === 0 || active.some(a => tags.includes(a))) {{
              card.style.display = 'block';
            }} else {{
              card.style.display = 'none';
            }}
          }});
        }});
      }});
    </script>
    """.format(cards="".join(out))


In [28]:
intro_html = _render_intro_html("""
<ul style="list-style-type: none; padding-left: 0; font-family: Arial, sans-serif; font-size:14px; line-height:1.6;">
  <li style="margin-bottom:6px;">🔗 Projektyta <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue" target="_blank">GITHUB</a></li>
  <li style="margin-bottom:6px;">🖼️ <a href="https://commons.wikimedia.org/wiki/Category:Stockholm%20Archipelago%20Trail" target="_blank">Bilder på leden</a> – Wikicommons - <a href="https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail&subcats=true&subcatdepth=4&cluster=false">Karta (tar tid)</a></li>
  <li style="margin-bottom:6px;">👣 Icke officiell <a href="https://www.facebook.com/groups/2875020699552247" target="_blank">FB grupp om leden av vandrare för vandrare</a></li>
  <li style="margin-bottom:6px;">📚 <a href="https://www.wikidata.org/wiki/Wikidata:WikiProject_Stockholm_Archipelago_Trail" target="_blank">Wikipedia projekt</a> / Umap: <a target=_blank href="https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true">NordSyd</a>, <a target=_blank href="https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true">Tälta</a></li>
  <li style="margin-bottom:6px;">🔗 Video om denna yta <a href="https://youtu.be/vy_746Wn4pc" target="_blank">youtube</a> / <a target="_blank" href="https://www.youtube.com/playlist?list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R">SAT spelista</a></li>
  <li style="margin-bottom:6px;">🔗 Samtal med: <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146" target="_blank">Tillväxtverket</a>, <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174" target="_blank">Naturvårdsverket</a> om dom brister vi ser.
</ul>
""")



In [29]:
import pandas as pd
from datetime import datetime
import time,os,sys,re
from pathlib import Path 
import shutil

OUTPUT_DIR   = "./output"  # ändra vid behov
PROJECT_NAME = "SAT_ALL_IN_ONE_142_3"
stamp        = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

print(stamp)


dash_html = make_sat_dashboard(
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        intro_html=intro_html,
        map_gallery=gallery,
        user_cases=user_cases
    )
print("✅ Dashboard with gallery =>", dash_html)

20251005_1221
Saved: output/SAT_ALL_IN_ONE_142_3_dashboard_20251005_1221.html
Updated: output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html
Current path: /Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/notebook
✅ Dashboard with gallery => output/SAT_ALL_IN_ONE_142_3_dashboard_20251005_1221.html


In [19]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-10-05 12:13:45
Total time elapsed: 06 minutes 26.53 seconds
